In [1]:
import pandas as pd
from Bio import Entrez

Entrez.email = 'kaedeito@student.ubc.ca'
Entrez.tool = 'Chlamydomonas-metal-toxicity'

In [2]:
references_df = pd.read_excel('../references/references.xlsx', sheet_name='list')
references_df.head()



,Checked?,included?,paper downloaded?,BioProject,Gene Expression Omnibus,DOI,description,"Notes (searched using ""Chlamydomonas"" ""metal"" ""toxicity"" ""SRA"")"
0,True,False,NaN,NaN,NaN,10.1128/AEM.00998-13,Transcriptome Sequencing (RNA-seq) Analysis of...,"Dana F. Simon, Rute F. Domingos. No SRAs avail..."
1,True,False,NaN,PRJNA799651,NaN,NaN,Insights from comparative transcriptome analys...,don’t include
2,True,False,0.0,SRP040767,NaN,10.1371/journal.pone.0107092,NaN,"control, AgNO3 and AgNP, metatranscriptome, th..."
3,True,True,1.0,PRJNA608616,NaN,10.1093/plphys/kiab375,Long-term acclimation to cadmium exposure reve...,"combo of control (Ctrl), short-term (ST), and ..."
4,True,True,1.0,PRJNA735693,GSE176268,10.1016/j.envpol.2021.117594,Rare earth elements show antagonistic interact...,"(5 replicates for each treatment: Ce, Tm, Y, M..."


In [3]:
bioprojects_df = references_df[references_df['included?'] == True]
bioprojects_list = bioprojects_df['BioProject'].tolist()
bioprojects_df.head()

,Checked?,included?,paper downloaded?,BioProject,Gene Expression Omnibus,DOI,description,"Notes (searched using ""Chlamydomonas"" ""metal"" ""toxicity"" ""SRA"")"
3,True,True,1.0,PRJNA608616,NaN,10.1093/plphys/kiab375,Long-term acclimation to cadmium exposure reve...,"combo of control (Ctrl), short-term (ST), and ..."
4,True,True,1.0,PRJNA735693,GSE176268,10.1016/j.envpol.2021.117594,Rare earth elements show antagonistic interact...,"(5 replicates for each treatment: Ce, Tm, Y, M..."
5,True,True,1.0,PRJNA335844,GSE84995,10.1016/j.watres.2017.05.020,Transcriptomic approach for assessment of the ...,"Valcea is control, Babeni is experiment"
6,True,True,1.0,PRJNA394256,NaN,10.1016/j.aquatox.2018.04.020,Comparison of genetic differences and transcri...,"strain is the same, methodology is the same, s..."
7,True,True,1.0,PRJNA576974,NaN,10.3389/fmicb.2020.01443,NaN,Control 1: SRR10269729 \nControl 2: SRR102697...


In [16]:
# search for SRAs associated with each BioProject
def get_sra_list():
    sra_list = []
    for bioproject in bioprojects_list:
        handle = Entrez.esearch(db='sra', term=bioproject, retmode='xml', rettype='uilist')
        record = Entrez.read(handle, validate=False)
        id_list: list[str] = record['IdList']
        handle.close()
        for id in id_list:
            entry = { 'BioProject': bioproject, 'SRA_ID': id }
            sra_list.append(entry)
    return sra_list

sra_list = get_sra_list()
sra_df = pd.DataFrame(sra_list)
sra_df.to_csv('../references/sra_list.csv', index=False)

sra_df['SRA_ID'].to_csv('../references/sra_id_list.txt', header=None, index=None)